In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# 让画图内置
%matplotlib inline

# 忽略版本号之类的warnings
import warnings
warnings.filterwarnings('ignore')
import os
from datetime import date

# 数据导入

In [ ]:
# 导入数据
# original_data = pd.read_csv('data\processed_data.csv')
# data = original_data.copy()
# data.shape

In [ ]:
# 导入数据
data = pd.read_csv('data\processed_data.csv')
data.shape

(42413557, 11)

In [ ]:
data['event_time'] = pd.to_datetime(data['event_time'])
data['category_code'] = data['category_code'].astype('category')
data['brand'] = data['brand'].astype('category')

data['event_day'] = data['event_time'].dt.date
data['event_hour'] = data['event_time'].dt.hour

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42413557 entries, 0 to 42413556
Data columns (total 13 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int64         
 3   category_id    int64         
 4   category_code  category      
 5   brand          category      
 6   price          float64       
 7   user_id        int64         
 8   category       object        
 9   sub_category   object        
 10  product_name   object        
 11  event_day      object        
 12  event_hour     int32         
dtypes: category(2), datetime64[ns](1), float64(1), int32(1), int64(3), object(5)
memory usage: 3.4+ GB


In [ ]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,category,sub_category,product_name,event_day,event_hour
0,2019-10-01 00:00:00,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,NaN,NaN,NaN,2019-10-01,0
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,appliances,environment,water_heater,2019-10-01,0
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,furniture,living_room,sofa,2019-10-01,0
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,computers,notebook,notebook,2019-10-01,0
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,electronics,smartphone,smartphone,2019-10-01,0


# 异常监测

## 异常访问量

In [ ]:
user_behavior = data.groupby(['user_id', 'event_type']).agg({'event_type': 'count'}).unstack().fillna(0)
user_behavior.columns = user_behavior.columns.droplevel()
user_behavior

event_type,cart,purchase,view
user_id,,,
33869381,0.0,0.0,1.0
64078358,0.0,0.0,1.0
183503497,0.0,0.0,1.0
184265397,0.0,0.0,6.0
195082191,0.0,0.0,1.0
...,...,...,...
566280663,0.0,0.0,2.0
566280676,0.0,0.0,1.0
566280697,0.0,0.0,1.0


### **总访问量异常**

In [ ]:
# 不下单的用户
user_behavior_0_purchase = user_behavior.query('purchase == 0')

# 浏览次数分布
user_view_count = user_behavior_0_purchase['view'].value_counts(normalize=True) *100

# user_view_count[user_view_count > 1]
user_view_count.sort_index(ascending=False)

view
7436.0     0.000037
4011.0     0.000037
2911.0     0.000037
2893.0     0.000037
2426.0     0.000037
            ...    
4.0        6.707718
3.0        9.585739
2.0       13.551914
1.0       26.089650
0.0        0.003663
Name: proportion, Length: 757, dtype: float64

In [ ]:
# pv数超过99.99%的用户,268个，min = 575 max = 7436 
ano_view_user = user_behavior_0_purchase[user_behavior_0_purchase['view'] >= user_behavior_0_purchase['view'].quantile(0.9999)] # 
ano_view_user.sort_values(by='view', ascending=False)

event_type,cart,purchase,view
user_id,,,
512475445,0.0,0.0,7436.0
512365995,2.0,0.0,4011.0
526731152,1.0,0.0,2911.0
512505687,1.0,0.0,2893.0
546270188,0.0,0.0,2426.0
...,...,...,...
515185376,4.0,0.0,577.0
512424032,3.0,0.0,575.0
513703754,0.0,0.0,575.0


### **单日访问量异常**

In [ ]:
user_behavior_0_purchase = user_behavior.query('purchase == 0')

# 没下过单的 用户的记录
user_records_0_purchase = data[data['user_id'].isin(user_behavior_0_purchase.index)]

user_records_0_purchase

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,category,sub_category,product_name,event_day,event_hour
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,appliances,environment,water_heater,2019-10-01,0
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,furniture,living_room,sofa,2019-10-01,0
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,computers,notebook,notebook,2019-10-01,0
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,electronics,smartphone,smartphone,2019-10-01,0
6,2019-10-01 00:00:08,view,17300353,2053013553853497655,NaN,creed,380.96,555447699,NaN,NaN,NaN,2019-10-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42413552,2019-10-31 23:59:58,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,electronics,camera,video,2019-10-31,23
42413553,2019-10-31 23:59:58,view,10800172,2053013554994348409,NaN,redmond,61.75,527322328,NaN,NaN,NaN,2019-10-31,23
42413554,2019-10-31 23:59:58,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,auto,accessories,player,2019-10-31,23
42413555,2019-10-31 23:59:59,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,electronics,clocks,clocks,2019-10-31,23


In [ ]:
user_daily_view_count = user_records_0_purchase.groupby(['user_id', 'event_day']).size()
user_daily_view_count

user_id    event_day 
33869381   2019-10-23    1
64078358   2019-10-13    1
183503497  2019-10-02    1
184265397  2019-10-04    4
           2019-10-15    2
                        ..
566280663  2019-10-31    2
566280676  2019-10-31    1
566280697  2019-10-31    1
566280780  2019-10-31    1
566280860  2019-10-31    1
Length: 5079259, dtype: int64

In [ ]:
# 每日访问量前 99.99% 的用户 496人， min = 179 , max = 1524 
ano_daily_view_user = user_daily_view_count[user_daily_view_count>user_daily_view_count.quantile(0.9999)]
ano_daily_view_user.sort_values(ascending=False)

user_id    event_day 
542048657  2019-10-22    1524
561163588  2019-10-23     738
513851612  2019-10-24     694
512475445  2019-10-30     640
561163588  2019-10-22     620
                         ... 
559223930  2019-10-11     179
560515816  2019-10-17     179
542485927  2019-10-06     179
543459553  2019-10-27     179
554748801  2019-10-26     179
Length: 496, dtype: int64

### 重复的

In [ ]:
ano_daily_view_user = ano_daily_view_user.index.droplevel(1).unique()
ano_daily_view_user # 4298

Index([430276841, 453363558, 454555218, 458662942, 463020196, 470651295,
       470911642, 472031105, 472278355, 483800649,
       ...
       566078787, 566121756, 566135499, 566145120, 566165785, 566170103,
       566179353, 566194918, 566209660, 566224639],
      dtype='int64', name='user_id', length=4298)

In [ ]:
# 单日view异常(4298) 和 月view异常(2693) 重复的用户数(1306)
ano_daily_view_user.isin(ano_view_user.index).sum()

1306

## **下单数异常**

### 用户 总下单数 异常

In [ ]:
# 下单的用户
user_order = user_behavior.query('purchase > 0')

# 下单数的分布
user_order_count = user_order['purchase'].value_counts(normalize=True)*100
user_order_count.sort_index(ascending=False)

purchase
321.0     0.000288
291.0     0.000288
242.0     0.000288
204.0     0.000288
193.0     0.000288
           ...    
5.0       2.119164
4.0       3.767019
3.0       7.452509
2.0      18.888966
1.0      62.143133
Name: proportion, Length: 132, dtype: float64

In [ ]:
# 下单数超过99.99%的用户, 有35人，min = 121 , max = 321
ano_order_user = user_order[user_order['purchase'] >= user_order['purchase'].quantile(0.9999)] # 44
ano_order_user

event_type,cart,purchase,view
user_id,,,
512386086,148.0,321.0,630.0
512511358,31.0,110.0,244.0
513117637,56.0,185.0,466.0
513320236,121.0,242.0,587.0
513322839,70.0,146.0,287.0
513784794,4.0,126.0,221.0
515384420,124.0,122.0,234.0
516214275,48.0,113.0,165.0
517728689,356.0,204.0,611.0


In [ ]:
ano_order_user.shape

(35, 3)

### 用户×商品 下单数异常

In [ ]:
order_info = data.query('event_type == "purchase"')

order_count_user_product = order_info.groupby(['user_id','product_id']).agg({
                                                      'brand':'unique',
                                                      'category_code':'unique',
                                                      'price':'count',
                                                      })
order_count_user_product['brand'] = order_count_user_product['brand'].apply(lambda x :x[0])
order_count_user_product['category_code'] = order_count_user_product['category_code'].apply(lambda x: x[0])
order_count_user_product.rename(columns={'price':'order_count'},inplace=True)
order_count_user_product

brand           category_code  order_count
user_id   product_id                                               
264649825 8500083     kiturami                     NaN            1
          8500084     kiturami                     NaN            1
303160429 5100443       garmin      electronics.clocks            1
340041246 2100099           lg    electronics.video.tv            1
          9300037           lg                     NaN            3
...                        ...                     ...          ...
566272569 12704161     pirelli                     NaN            1
          12708874        toyo                     NaN            1
566274637 1005136        apple  electronics.smartphone            1
566276996 12702883    cordiant                     NaN            1
566278294 1005144        apple  electronics.smartphone            1

[558484 rows x 3 columns]

In [ ]:
# 每个产品下单数异常 的用户 超过99.99% ， 共45人 min = 42 max = 248
threshold_count = order_count_user_product['order_count'].quantile(0.9999)
ano_order_user_product = order_count_user_product.query('order_count > @threshold_count')
ano_order_user_product

brand                           category_code  \
user_id   product_id                                                     
512386086 1004227        apple                  electronics.smartphone   
          1004767      samsung                  electronics.smartphone   
          1801555           lg                    electronics.video.tv   
          4201541        artel  appliances.environment.air_conditioner   
512409624 1004249        apple                  electronics.smartphone   
512560473 22400121     m-audio            electronics.audio.microphone   
512681805 1004958       xiaomi                  electronics.smartphone   
512786896 1307345         acer                      computers.notebook   
512935711 5100572        apple                      electronics.clocks   
513117637 1005117        apple                  electronics.smartphone   
513279092 1004249        apple                  electronics.smartphone   
513320236 1004767      samsung                  electronics.smartphone   
          1801555           lg                    electronics.video.tv   
513717990 1005119        apple                  electronics.smartphone   
513784794 1201397        apple                      electronics.tablet   
514028365 31500231    luminarc                                     NaN   
514179041 4804056        apple             electronics.audio.headphone   
514948520 3701084       xiaomi           appliances.environment.vacuum   
514950756 1005144        apple                  electronics.smartphone   
515384420 1005135        apple                  electronics.smartphone   
          1005144        apple                  electronics.smartphone   
516214275 1004209      samsung                  electronics.smartphone   
517728689 1004766      samsung                  electronics.smartphone   
          1004835      samsung                  electronics.smartphone   
518899896 1004566       huawei                  electronics.smartphone   
519250600 1005130        apple                  electronics.smartphone   
519267944 1005140        apple                  electronics.smartphone   
519322381 2201036        canon                                     NaN   
523974502 5100563      samsung                      electronics.clocks   
526933626 1004833      samsung                  electronics.smartphone   
530206193 4804056        apple             electronics.audio.headphone   
530224131 1004720       huawei                  electronics.smartphone   
530834332 1005074      samsung                  electronics.smartphone   
532499743 1005133        apple                  electronics.smartphone   
535658024 1002544        apple                  electronics.smartphone   
538216048 1004237        apple                  electronics.smartphone   
539479985 5100566        apple                      electronics.clocks   
541510103 1004833      samsung                  electronics.smartphone   
          1004856      samsung                  electronics.smartphone   
543312954 1004767      samsung                  electronics.smartphone   
          1004856      samsung                  electronics.smartphone   
545925192 1005072      samsung                  electronics.smartphone   
547399783 1005121        apple                  electronics.smartphone   
549109608 1004856      samsung                  electronics.smartphone   
549165455 1004958       xiaomi                  electronics.smartphone   
551361102 1002528        apple                  electronics.smartphone   
553431815 1005107        apple                  electronics.smartphone   
555856365 12706441      nokian                                     NaN   
557935258 1004161       xiaomi                  electronics.smartphone   
558762835 1004833      samsung                  electronics.smartphone   
560854183 1004739       xiaomi                  electronics.smartphone   
563791543 1004833      samsung                  electronics.smartphone   
564068124 1004767      samsun

In [ ]:
# 下单数异常的用户(35) 和 不同产品下单数异常的用户(45) 重合的用户数(21)
ano_order_user_product.index.droplevel(1).unique().isin(ano_order_user.index).sum() # 21

21

In [ ]:
ano_order_user_product['brand'].value_counts()[:10]

brand
apple       21
samsung     18
xiaomi       5
lg           2
huawei       2
artel        1
m-audio      1
acer         1
luminarc     1
canon        1
Name: count, dtype: int64

In [ ]:
ano_order_user_product['category_code'].value_counts()[0:10]

category_code
electronics.smartphone                    39
electronics.clocks                         3
electronics.video.tv                       2
electronics.audio.headphone                2
appliances.environment.air_conditioner     1
electronics.audio.microphone               1
computers.notebook                         1
electronics.tablet                         1
appliances.environment.vacuum              1
Name: count, dtype: int64